
# Part 1 — Pima Diabetes: Feature Selection (5-Fold CV, with Imputation)

This notebook follows the assignment and uses **zero-as-missing imputation** in the pipelines to avoid data leakage.



## Objective
Compare three feature selection methods and determine which yields the highest test accuracy:

- `VarianceThreshold`
- `SelectKBest(chi²)` *(requires non-negative features)*
- `RFE(LogisticRegression)`

We use **Stratified 5-Fold CV (Cross-Validation)** and report mean ± std accuracy. We also inspect which features were selected.


## Setup & Imports

In [17]:

import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

pd.set_option("display.precision", 2)
pd.set_option("display.max_colwidth", 90)



## Configuration
- Set the dataset path
- Choose `k` for the number of features in `SelectKBest` and `RFE`.
- Set `VAR_THRESHOLD` for the variance threshold.
- Set `N_FOLDS` for cross-validation with `StratifiedKFold`.


In [18]:

DATA_PATH = "Datasets/pima-indians-diabetes.csv"

COLUMN_NAMES = [
    "Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin",
    "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"
]

K_FEATURES = 5  # for SelectKBest and RFE
VAR_THRESHOLD = 0.05 # Variance threshold
N_FOLDS = 5

CLF_KWARGS = dict(solver="liblinear", max_iter=1000, penalty="l2")



## Step 1 — Load and Inspect Data
Load the CSV (skipping commented header lines), separate features and target, and check basic stats.


In [19]:

# Load
df = pd.read_csv(DATA_PATH, header=None, names=COLUMN_NAMES, comment="#")

# Separate features/target
y = df["Outcome"].copy()
X = df.drop(columns=["Outcome"]).copy()

# Inspect
print("Shape:", df.shape)
display(df.head())
display(df.isna().sum().to_frame("missing_count"))

feature_ranges = pd.DataFrame({
    "min": X.min(),
    "max": X.max(),
    "mean": X.mean(),
    "std": X.std(ddof=0)
})
display(feature_ranges)


Shape: (768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.63,50,1
1,1,85,66,29,0,26.6,0.35,31,0
2,8,183,64,0,0,23.3,0.67,32,1
3,1,89,66,23,94,28.1,0.17,21,0
4,0,137,40,35,168,43.1,2.29,33,1


,missing_count
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


,min,max,mean,std
Pregnancies,0.00,17.00,3.85,3.37
Glucose,0.00,199.00,120.89,31.95
BloodPressure,0.00,122.00,69.11,19.34
SkinThickness,0.00,99.00,20.54,15.94
Insulin,0.00,846.00,79.80,115.17
BMI,0.00,67.10,31.99,7.88
DiabetesPedigreeFunction,0.08,2.42,0.47,0.33
Age,21.00,81.00,33.24,11.75



## Handle Invalid Zeros → NaN
In these columns, zeros are invalid/implausible and treated as missing:  
`Glucose`, `BloodPressure`, `SkinThickness`, `Insulin`, `BMI`.


In [20]:

zero_as_missing_cols = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# Show zero counts before conversion
zero_counts = (X[zero_as_missing_cols] == 0).sum().to_frame("zero_count_before")
display(zero_counts)

# Replace zeros with NaN
X_imp = X.copy()
for c in zero_as_missing_cols:
    X_imp.loc[X_imp[c] == 0, c] = np.nan

# Show missing counts after conversion
missing_after = X_imp.isna().sum().to_frame("missing_after_zero_to_nan")
display(missing_after)


,zero_count_before
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,11


,missing_after_zero_to_nan
Pregnancies,0
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,11
DiabetesPedigreeFunction,0
Age,0



## Step 2 — Cross-Validation & Metric
Use **StratifiedKFold (5 folds)** and **accuracy** as the evaluation metric.


In [21]:

cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
scoring = "accuracy"



## Step 3 — Define Pipelines (with Imputation)
Each pipeline starts with `SimpleImputer(strategy='median')` to avoid leakage **inside** cross-validation:

1. `VarianceThreshold → StandardScaler → LogisticRegression`  
2. `MinMaxScaler → SelectKBest(chi², k=K_FEATURES) → LogisticRegression` *(chi² needs non-negative features)*  
3. `StandardScaler → RFE(LogisticRegression, k=K_FEATURES) → LogisticRegression`


In [22]:

pipe_var = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("var", VarianceThreshold(threshold=VAR_THRESHOLD)),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(**CLF_KWARGS))
])

pipe_kbest = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("minmax", MinMaxScaler()),
    ("kbest", SelectKBest(score_func=chi2, k=K_FEATURES)),
    ("clf", LogisticRegression(**CLF_KWARGS))
])

pipe_rfe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("rfe", RFE(estimator=LogisticRegression(**CLF_KWARGS), n_features_to_select=K_FEATURES, step=1)),
    ("clf", LogisticRegression(**CLF_KWARGS))
])

pipelines = {
    "VarianceThreshold + LR": pipe_var,
    f"SelectKBest(chi2, k={K_FEATURES}) + LR": pipe_kbest,
    f"RFE(LR, k={K_FEATURES}) + LR": pipe_rfe
}



## Step 4 — Evaluate Each Pipeline (5-Fold CV)
We report the mean and standard deviation of accuracy and the per-fold scores.


In [23]:

rows = []
for name, pipe in pipelines.items():
    scores = cross_val_score(pipe, X_imp, y, cv=cv, scoring=scoring)
    rows.append({
        "Pipeline": name,
        "Mean Accuracy": scores.mean(),
        "Std Accuracy": scores.std(),
        "Fold Accuracies": np.round(scores, 2).tolist()
    })

results_df = pd.DataFrame(rows).sort_values("Mean Accuracy", ascending=False).reset_index(drop=True)
display(results_df)

best = results_df.iloc[0]
BEST_PIPELINE_NAME = best["Pipeline"]
BEST_MEAN_ACC = float(best["Mean Accuracy"])
BEST_STD_ACC = float(best["Std Accuracy"])


,Pipeline,Mean Accuracy,Std Accuracy,Fold Accuracies
0,VarianceThreshold + LR,0.77,0.02,"[0.77, 0.8, 0.78, 0.76, 0.76]"
1,"RFE(LR, k=5) + LR",0.77,0.02,"[0.78, 0.8, 0.78, 0.76, 0.75]"
2,"SelectKBest(chi2, k=5) + LR",0.77,0.02,"[0.75, 0.81, 0.79, 0.75, 0.76]"



## Step 5 — Inspect Selected Features
Fit each selector on the **full dataset (with imputation)** to list which features were kept.  
We also show chi² scores and RFE rankings.


In [24]:

# Fit simple imputer to full feature matrix for inspection
imputer_full = SimpleImputer(strategy="median")
X_full = pd.DataFrame(imputer_full.fit_transform(X_imp), columns=X_imp.columns)

# 1) VarianceThreshold
var_selector = VarianceThreshold(threshold=VAR_THRESHOLD)
var_selector.fit(X_full, y)
var_kept = X_full.columns[var_selector.get_support()].tolist()

# 2) SelectKBest(chi2)
minmax_full = MinMaxScaler()
X_full_nonneg = pd.DataFrame(minmax_full.fit_transform(X_full), columns=X_full.columns)
kbest_selector = SelectKBest(score_func=chi2, k=K_FEATURES)
kbest_selector.fit(X_full_nonneg, y)
kbest_kept = X_full_nonneg.columns[kbest_selector.get_support()].tolist()
chi2_scores = pd.Series(kbest_selector.scores_, index=X_full_nonneg.columns).sort_values(ascending=False)

# 3) RFE(LogisticRegression)
scaler_full = StandardScaler()
X_full_std = pd.DataFrame(scaler_full.fit_transform(X_full), columns=X_full.columns)
rfe_selector = RFE(estimator=LogisticRegression(**CLF_KWARGS), n_features_to_select=K_FEATURES, step=1)
rfe_selector.fit(X_full_std, y)
rfe_kept = X_full_std.columns[rfe_selector.support_].tolist()
rfe_ranking = pd.Series(rfe_selector.ranking_, index=X_full_std.columns).sort_values()

# Collate and display
selected_features_df = pd.DataFrame([
    {"Selector": f"VarianceThreshold (threshold={VAR_THRESHOLD})",
     "Selected/Kept Features": ", ".join(var_kept) if len(var_kept) else "(none removed)"},
    {"Selector": f"SelectKBest(chi2, k={K_FEATURES})",
     "Selected/Kept Features": ", ".join(kbest_kept)},
    {"Selector": f"RFE(LogisticRegression, k={K_FEATURES})",
     "Selected/Kept Features": ", ".join(rfe_kept)},
])
display(selected_features_df)

display(chi2_scores.to_frame("chi2_score").reset_index(names="feature"))
display(rfe_ranking.to_frame("RFE_ranking").reset_index(names="feature"))


,Selector,Selected/Kept Features
0,VarianceThreshold (threshold=0.05),"Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunc..."
1,"SelectKBest(chi2, k=5)","Pregnancies, Glucose, BMI, DiabetesPedigreeFunction, Age"
2,"RFE(LogisticRegression, k=5)","Pregnancies, Glucose, BMI, DiabetesPedigreeFunction, Age"


,feature,chi2_score
0,Glucose,14.34
1,Age,8.21
2,Pregnancies,6.56
3,BMI,5.06
4,DiabetesPedigreeFunction,2.76
5,Insulin,2.26
6,SkinThickness,1.35
7,BloodPressure,0.65


,feature,RFE_ranking
0,Pregnancies,1
1,Glucose,1
2,DiabetesPedigreeFunction,1
3,BMI,1
4,Age,1
5,BloodPressure,2
6,Insulin,3
7,SkinThickness,4



## Step 6 — Reflection

**Which method achieved the highest test accuracy?**  
Use the comparison table above; in this run it is captured by variables in the evaluation cell (`BEST_PIPELINE_NAME`, `BEST_MEAN_ACC`, `BEST_STD_ACC`). However, with this configuration there was no difference in test accuracy.

**Why might their performances differ?**  
- **SelectKBest(chi²)** is **univariate** and may remove features that are weak individually but useful jointly.  
- **RFE(LogisticRegression)** is **model-aware** but can be sensitive to `k` and to small sample sizes; it may overfit during selection.  
- **VarianceThreshold + Logistic Regression** retains all potentially informative variance; with scaling and regularization, LR can benefit from correlated features and avoid discarding signal.

**Which method is most suitable for this dataset, and why?**  
With proper imputation, **regularized Logistic Regression with minimal filtering** (VarianceThreshold) is often a strong baseline on Pima. If dimensionality reduction is desired, prefer a **model-aware** approach like RFE and tune `k` via (nested) CV, but verify gains via cross-validation.
